In [2]:
import sys
sys.path.append("../src/")
sys.path.append("../")



import torch
from bert import *
from feature_data import *
from multiprototype import *
from models import *
from utils import *
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

In [3]:
bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmp8mnx01zy
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [40]:
"""
take a look at the eval words
"""

"""
parse into a dataset with id, orig_sent, and swap_sent
"""

pd.set_option('display.max_colwidth', None)

filename = '../data/processed/swarm data - aggregated.csv'


    
eval_words = pd.read_csv(filename)

print(len(eval_words))
print(eval_words.groupby(['verb', 'preposition']).ngroups)

print(eval_words[['verb', 'preposition']].value_counts(ascending=True).reset_index(name='count'))
eval_words.head(20)

844
6
   verb  preposition  count
0  creep     with        7 
1  creep       in       11 
2  swarm       in       63 
3   swim     with      146 
4  swarm     with      172 
5   swim       in      445 


,list_id,source,sentence,verb,preposition
0,1,COCA:2019:FIC\nSouthernRev,"I wondered what kind of mind was lurking behind that white smile and those wide brown eyes, Felicia was thoughtful in her way, and she had Freud's theory of ambivalence on her side: Was there a contradiction between doting on your wife and plotting to kill her? # The world swarmed with possibilities that might surprise others but did not startle Felicia, and all possibilitiesmade an equal claim on her interest. This had a leveling effect on her conversation. She could take up any topic -- the new dining room chairs she was thinking about, a once-favored restaurant now in decline",swarm,with
1,2,COCA:2019:FIC\nNewYorker,"had volunteered. # Vollie led Heflin toward a cocktail lounge. Or maybe it was the other way around: Heflin led Vollie. Or the other, other way: both of them led by the red neon figure of a Martini glass suspended above the establishment's door. The place inside swarmed with marines -- you'd think an invasion was on -- skinny pimpled white black, laughing idiots eager to get mowed down to the ankles, reaped like corn. Heflin had gone away again, because here he was coming back with two fresh cans. "" Give me your life, """,swarm,with
2,3,COCA:2019:ACAD\nCato Journal,"by Erhard (1958). Important characteristics of the prewar and war economic system prevailed. # 2Erhard (1958: 13) cites Jacques Rueff and André Piettre as follows: "" The black market suddenly disappeared. Shop windows were full of goods; factory chimneys were smoking; and the streets swarmed with lorries. Everywhere the noise of the new buildings going up replaced the deathlysilence of the ruins. If the state of recovery was a surprise, its swiftness was even more so. In all sectors of economic life it began as the clocks struck on the day of currency reform.",swarm,with
3,4,COCA:2018:FIC\nNewEnglandRev,"through the books I'd read and films I'd seen at the French and American cultural institutes was enough for me to imagine their strange lives in that forbidden paradise. I pictured diplomatic wives fox-trotting around night-lit pools , whirling in the arms of their lovers' white tuxedos, while smoke-filled parlors swarmed with Greek merchants and diamond smugglers betting and losing their millions at poker; andthere in the garden, under a straw gazebo, young colonels who were distant cousins of the president conferred, plotting the next coup d'?? tat. And, overlooking Kiriri Hill loomed Saint-Esprit College,",swarm,with
4,5,COCA:2018:MAG\nVanity Fair,"of state, under his arm. She signed, "" Best wishes, Hillary, "" and as she handed it back with a come-hither wink, said, "" It's a complicated world, is n't it ? '' # After speaking at a bowling alley in Adel, Hillary was so swarmed with a group of teachers ('' I hope you get an excused absence today! "") that she grabbed my phone right out of my hand to pose for a selfie. Huma Abedin whispered in Hillary's ear, "" That's Amy's, "" and Hillary handed it back so",swarm,with
5,6,COCA:2016:FIC\nLiterary Review,"itself for hundreds of years. "" He marveled at the dirt under his boots-what used to be an entire tree, a speck of mountain, dark matter from the center of the earth. # Once, on a hike to a grove of dogwoods, we found the head of a deer swarmed with ants, picked clean to the bone except for its one antler, stillpartially covered in velvet. # Thomas had wondered at the sight of it. What had killed it? Where was the rest of its body? Where was its other antler? # Days later, he was",swarm,with
6,7,COCA:2016:FIC\nFantasySciFi,"continually by some fluid invisible hand that gave it animation, keeping it in constant, shimmering flux. The colors of objects barely stayed within the lines that sought to contain them, as if the inks with which the world was painted were trembling, blurring, running free. Hewell's flesh swarmed with tiny etched lozenges and diamond-shaped pores, his skin but a net of finestmesh that barely held his 

In [46]:
def label_lemma(row):
    if row.verb == 'swarm':
        return 'swarmed'
    elif row.verb == 'swim':
        return 'swam'
    elif row.verb == 'creep':
        return 'creeping'

eval_words['word_form'] = eval_words.apply(lambda row: label_lemma(row), axis=1)
eval_words

,list_id,source,sentence,verb,preposition,word_form
0,1,COCA:2019:FIC\nSouthernRev,"I wondered what kind of mind was lurking behind that white smile and those wide brown eyes, Felicia was thoughtful in her way, and she had Freud's theory of ambivalence on her side: Was there a contradiction between doting on your wife and plotting to kill her? # The world swarmed with possibilities that might surprise others but did not startle Felicia, and all possibilitiesmade an equal claim on her interest. This had a leveling effect on her conversation. She could take up any topic -- the new dining room chairs she was thinking about, a once-favored restaurant now in decline",swarm,with,swarmed
1,2,COCA:2019:FIC\nNewYorker,"had volunteered. # Vollie led Heflin toward a cocktail lounge. Or maybe it was the other way around: Heflin led Vollie. Or the other, other way: both of them led by the red neon figure of a Martini glass suspended above the establishment's door. The place inside swarmed with marines -- you'd think an invasion was on -- skinny pimpled white black, laughing idiots eager to get mowed down to the ankles, reaped like corn. Heflin had gone away again, because here he was coming back with two fresh cans. "" Give me your life, """,swarm,with,swarmed
2,3,COCA:2019:ACAD\nCato Journal,"by Erhard (1958). Important characteristics of the prewar and war economic system prevailed. # 2Erhard (1958: 13) cites Jacques Rueff and André Piettre as follows: "" The black market suddenly disappeared. Shop windows were full of goods; factory chimneys were smoking; and the streets swarmed with lorries. Everywhere the noise of the new buildings going up replaced the deathlysilence of the ruins. If the state of recovery was a surprise, its swiftness was even more so. In all sectors of economic life it began as the clocks struck on the day of currency reform.",swarm,with,swarmed
3,4,COCA:2018:FIC\nNewEnglandRev,"through the books I'd read and films I'd seen at the French and American cultural institutes was enough for me to imagine their strange lives in that forbidden paradise. I pictured diplomatic wives fox-trotting around night-lit pools , whirling in the arms of their lovers' white tuxedos, while smoke-filled parlors swarmed with Greek merchants and diamond smugglers betting and losing their millions at poker; andthere in the garden, under a straw gazebo, young colonels who were distant cousins of the president conferred, plotting the next coup d'?? tat. And, overlooking Kiriri Hill loomed Saint-Esprit College,",swarm,with,swarmed
4,5,COCA:2018:MAG\nVanity Fair,"of state, under his arm. She signed, "" Best wishes, Hillary, "" and as she handed it back with a come-hither wink, said, "" It's a complicated world, is n't it ? '' # After speaking at a bowling alley in Adel, Hillary was so swarmed with a group of teachers ('' I hope you get an excused absence today! "") that she grabbed my phone right out of my hand to pose for a selfie. Huma Abedin whispered in Hillary's ear, "" That's Amy's, "" and Hillary handed it back so",swarm,with,swarmed
5,6,COCA:2016:FIC\nLiterary Review,"itself for hundreds of years. "" He marveled at the dirt under his boots-what used to be an entire tree, a speck of mountain, dark matter from the center of the earth. # Once, on a hike to a grove of dogwoods, we found the head of a deer swarmed with ants, picked clean to the bone except for its one antler, stillpartially covered in velvet. # Thomas had wondered at the sight of it. What had killed it? Where was the rest of its body? Where was its other antler? # Days later, he was",swarm,with,swarmed
6,7,COCA:2016:FIC\nFantasySciFi,"continually by some fluid invisible hand that gave it animation, keeping it in constant, shimmering flux. The colors of objects barely stayed within the lines that sought to contain them, as if the inks with which the world was painted were trembling, blurring, running free. Hewell's flesh swarmed with tiny etched lozenges and diamond-shaped por

In [36]:
# creep = eval_words[eval_words['verb'] == 'creep']
# creep
# creep.to_csv('../data/processed/creep_tokens.csv')

In [43]:
# load best binder model
#model = torch.load('../trained_models/main_82b2e_00003_3_clusters=5,embedding_type=bert,model=modabs,mu2=0.1,mu3=1e-07,mu4=5,nnk=3,train_data=binder_2022-10-13_20-50-01')
#model = torch.load('../trained_models/model.plsr.mc_rae_real.allbuthomonyms.5k.100components.500max_iters')
model = torch.load('../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters')


In [44]:
feature_map = model.feature_norms.feature_map
feature_labels = [str(feature_map.get_object(i)) for i in range(0, len(feature_map))]
print(feature_labels[:10])


['desert', 'give', 'leave', 'up', 'withdraw', 'belly', 'body', 'middle', 'muscle', 'organ']


In [48]:
eval_words_minus_swim = eval_words[eval_words['verb'] != 'swim']
len(eval_words_minus_swim)

253

In [49]:
"""
make a tidy dataseet of metaphor predictions for Kyle
"""

sentence_ids =[]
verbs = []
prepositions = []
features = []
values = []


predictions = []
for index, row in eval_words_minus_swim.iterrows():

    print(row.verb)
    print(row.sentence)
    prediction = model.predict_in_context(row.word_form, row.sentence, bert)
    i=0
    for value in prediction:
        sentence_ids.append(index)
        verbs.append(row.verb)
        prepositions.append(row.preposition)
        features.append(feature_labels[i])
        values.append(value)
        i+=1
        
    
tidy_df = pd.DataFrame.from_records(
    {"sentence_id": sentence_ids,
    "verb": verbs, 
    "preposition": prepositions, 
    "feauture": features,
    "value": values,
    }
)
tidy_df.to_csv('../data/processed/swarm_predictions_buchanan_minus_swim_tidy.csv')

tidy_df.head(5)

swarm
I wondered what kind of mind was lurking behind that white smile and those wide brown eyes, Felicia was thoughtful in her way, and she had Freud's theory of ambivalence on her side: Was there a contradiction between doting on your wife and plotting to kill her? # The world swarmed with possibilities that might surprise others but did not startle Felicia, and all possibilitiesmade an equal claim on her interest. This had a leveling effect on her conversation. She could take up any topic -- the new dining room chairs she was thinking about, a once-favored restaurant now in decline
swarm
had volunteered. # Vollie led Heflin toward a cocktail lounge. Or maybe it was the other way around: Heflin led Vollie. Or the other, other way: both of them led by the red neon figure of a Martini glass suspended above the establishment's door. The place inside swarmed with marines -- you'd think an invasion was on -- skinny pimpled white black, laughing idiots eager to get mowed down to the ankles

swarm
people had done for centuries before them. # For half an hour they endured the discomfort, and they were rewarded for it. Just as Hairan had predicted, a pool of water was inside a hollow in the sand. It wasn't much-barely enough to last the day -- and it swarmed with insects. But tomorrow was a new day, and it would bring asmuch hope as any other. The women kneeled to collect what little water there was, straining it through the gauze of their head veils to purify it. # Driven by a premonition that there was more to
swarm
all of my meals here. # Brown and Le certainly take risks with their menu, giving the food an edgy playfulness, but inevitably not all of them succeed. The hot dog ($5) served simply with German mustard stands up just fine on its own, but on a table swarmed with exciting dishes like the uni fettuccine and the buttery Maine lobster roll ($14) it looks like a waste of precious real estate. # Though the drink menu is n't the deepest in town, Octopus Bar sports a f

swarm
a family into a maze of chain-link pens filled with construction machinery. The family of three held hands as they fled among cranes and bulldozers. They looked like the image on the yellow freeway signs that depicted a family of running migrants to alert drivers to the fact that the roads around here swarmed with frightened, exhausted pedestrians who got run over in gory and spectacular ways.# Unlike the girl in the freeway sign, though, the little girl he chased did not wear pigtails, but rather ribbons in her hair and a silver party dress with a jeans jacket over it. For
swarm
From my position, which was far above the level of the town, I could perceive its every nook and corner, as if delineated on a map. The streets seemed innumerable, and crossed each other irregularly in all directions, but were rather long winding alleys than streets, and absolutely swarmed with inhabitants. The houses were wildly picturesque. On every hand was a wildernessof balconies, of verandas, of mi

swarm
see the sky meeting the sea on one side and the sky meeting the land on the other. So there you are among all those boundaries, right at the place where life climbed out of the sea. It has lots of distance for staring off into. For me, beaches always swarmed with past loves and past moments, vivid memories surfacing from the water and storminginland like movie Vikings in their horned helmets and scuba gear, waving their battle-axes and tommy guns. (Well, all right, that's a scene from Admiral Nelson on Iwo Jima, but every former fourteen-year-old
swarm
she give him a kiss! " Lady Hearst gazed anxiously at Lucilla. But the younger lady did not appear at all outraged. A faint, very secretive smile touched her lips. For a moment, Lucilla 's rather plain features assumed a very attractive and somewhat sensual look. The gathering swarmed with excited whispers. More people tried to crowd into the narrow entryway, manycraning their necks to... Continues...                     ##1030712 

swarm
So convinced was the Hungerkunstler of the amulet's uncanny powers that she pressed Baconfield for others. As it turned out the fellah lived in a hovel which straddled a tomb; it sufficed that he lift a stone from a hole in the floor to enter into the mastaba of a corpse which swarmed with priceless scarabs of opal, peridot, fayruz, onyx, and coraline.Some had the faces of rams, others of bulls, others had human faces. All of these were swallowed and digested by the Hungerkunstler. When a colossal scarabaeus sacer was uncovered by Baconfield himself as he rooted
swarm
armed humans troop by without a second look. Lights went on for Llenor, winking out behind her. Doors dilated. She loved the way the ship obeyed, tracking her movements, anticipating her needs. She told the cargo lift to take them down to the keel. The main hold swarmed with Bug Warriors, jammed muzzle to mandible to make room for a mobile pressurevault with a blast-proof lock. The usual eight-legged horrors were ba

...

swarm
moment, the only sound in the garage was the humming of flies as Jane considered every urban legend she'd ever heard about stolen organs. Then she focused on the covered garbage can in the far corner. As she approached it, the stench of putrefaction grew even stronger, and flies swarmed in a hungry cloud. Grimacing, she lifted the edge of the lid.One quick glance was all she could stomach before the smell made her back away, gagging. # " I take it you found them, " said Maura. # " Yeah, " muttered Jane. " At
swarm
this man would protect me; they knew there was nothing he would n't do to accomplish it. And that thing I felt welling up inside me, I feared the name of it was love. Command had declared victory in May, but soon as the marines pulled out, the insurgents swarmed in like locusts. By November, intel ligence estimated their numbers at 4,000.And these were n't your weekend jihadis, sprinting through alleyways w ith ski masks and bor rowed Kalashnikovs. These were m en who'd

swarm
phrases in our notebooks and wrapped these images around our hearts, the dragonflies appeared. As we answered the knocking doors of our souls, walked toward that voice that has called us, quietly and persistently, all of our lives, to write and claim our places as true artists, they swarmed in gentle circles over our heads. # We looked up from our notebooks andremarked about the magic of that particular moment. Indeed it was. The dragonflies never landed, never bothered us in any way. They did, however, perform a dragonfly ballet to the music that only a
swarm
# Your obviously a troublemaker and the site would be better without you. Do n't bother replying and quit harrasing the others " conservative " # Conan on November 12, 2012 at 10:51 PM # How long have you been commenting here? Since the last open registration when all the Romney ops swarmed in? I've been commenting here for the past 4 years, and I've been pretty consistent in my opinions. I'm a " troublemaker " in that I'm 

swarm
many informal hiring halls of a new city. For a few months, there were 300 or 400 a morning, a land-office business in soft drinks and snacks for the gas station. Now it's down to a hundred or so a day. No one knows how many Latin Americans have swarmed in since Katrina. One estimate guesses at least 100,000 in the Gulf region.Last October, Mayor Ray Nagin complained that his city was being overrun by Mexicans. In the 2000 census New Orleans was 3 percent Hispanic. Now one sees more brown faces than black. But then no one
swarm
a deputy after a suspected drug deal on Monday. And they warn that he is likely armed and dangerous now. Joining us with the very latest, live on the phone, is spokesman Jim Solomons from the Orange County , Florida, sheriffs department. Jim, first of all, tell us why deputies swarmed in on Mr. Forbes to begin with? JIM SOLOMONS, ORANGE COUNTY, FLORIDA, SHERIFFS OFFICE: Well, it wasnt a swarm in. It was a deputy on routine patrol that saw what he believed 

swarm
and ordered federal agents to shoot to kill. One FBI SWAT team member later told Justice department officials that he remembered the rules as, " If you see' em, shoot' em. " Four hundred government agents, armed with night-vision scopes, automatic weapons, and sniper rifles, swarmed in the mountains around the cabin. # The next day, August 22,Randy Weaver walked from his cabin to the little shack where his son's body lay. As Weaver lifted the latch on the shack's door, he was shot from behind by FBI sharpshooter Lon Horiuchi. He
swarm
, as they looked closer, they noticed the recent signs of a body on the dirty sheet. Once again they practically trod on the woman's heels and drank in her perfume and image: they felt themselves protected from the squalor of the place, as if safe from the powers that swarmed in its atmosphere. The bedroom was roughly as big as the other rooms,but to the men it seemed much bigger. And when at last they entered the yard and regained the sky and air, 

,feauture,preposition,sentence_id,value,verb
0,desert,with,0,-0.033,swarm
1,give,with,0,0.017,swarm
2,leave,with,0,-0.007,swarm
3,up,with,0,0.098,swarm
4,withdraw,with,0,-0.005,swarm


In [50]:
len(tidy_df)
tidy_df.head(100)


,feauture,preposition,sentence_id,value,verb
0,desert,with,0,-3.272e-02,swarm
1,give,with,0,1.653e-02,swarm
2,leave,with,0,-6.585e-03,swarm
3,up,with,0,9.753e-02,swarm
4,withdraw,with,0,-4.632e-03,swarm
5,belly,with,0,1.030e-02,swarm
6,body,with,0,1.840e-02,swarm
7,middle,with,0,-4.277e-02,swarm
8,muscle,with,0,5.899e-03,swarm
9,organ,with,0,2.372e-02,swarm


In [62]:
len(model.word_indexer)
model.kd_tree.n/5

AttributeError: 'FeatureClassifier' object has no attribute 'word_indexer'

### This is the static analysis

when we see feature values for subjects, objects, and then swapped subjects, and swapped objects.

Should have a results table 65 X 4

For example, here is a sentence

Other shops around this city have MUCH NICER and more TRANSPARENT owners .\n

subject: shops
object: owners

And here is a swapped sentence

Other owners around this city have MUCH NICER and more TRANSPARENT shops .\n

subject: owners
object: shops

You can look at these four values separately, and that is what I will do. 


In [63]:
df.head(1)

,id,sent,subject,object,swapped?
0,reviews-211797-0020\n,Other shops around this city have MUCH NICER and more TRANSPARENT owners .\n,shops,owners,False


In [64]:
#print(len(df))
#df.to_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')

In [65]:
ids = []
sent = []
subject = []
obj = []
swapped = []
condition = []
feature = []
predicted_value = []

feature_labels = list(features)


for index, row in df.iterrows():
        
    # predict features in context
#     singular = row.word
#     past = singular+"d"
#     try:
#         prediction = model.predict_in_context(singular, row.sentence, bert)
#         word = singular
#     except:
#         prediction = model.predict_in_context(past, row.sentence, bert)
#         word = past

    print(row.sent)
    print(row.subject)

    i = 0
    subject_prediction = model.predict_in_context(row.subject, row.sent, bert)
    #print(len(subject_prediction))
    #print(len(feature_labels))
    for value in subject_prediction:
        #print(feature_labels[i])
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("subject")
        feature.append(feature_labels[i])
        predicted_value.append(value)
        i+=1
    
    i=0
    print(row.object)
    object_prediction = model.predict_in_context(row.object, row.sent, bert)
    for value in object_prediction:
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("object")
        feature.append(feature_labels[i])
        predicted_value.append(value)
        i+=1
    
tidy_df = pd.DataFrame.from_records(
    {"id": ids,
    "subject": subject, 
    "object": obj, 
    "swapped?": swapped, 
    "condition": condition, 
    "feature": feature, 
    "predicted_value": predicted_value}
)
tidy_df.to_csv('../data/processed/swapped_subject_and_object_predictions_ffnn.csv')
 

#cols = ["id", "subject", "object", "swapped?", "condition", "feature", "predicted_value"]
#tidy_df = pd.DataFrame([ids, subject, obj, swapped, feature, predicted_value], columns = cols)
#tidy_df.to_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')
tidy_df.head(5)

Other shops around this city have MUCH NICER and more TRANSPARENT owners .

shops
owners
Other owners around this city have MUCH NICER and more TRANSPARENT shops .

owners
shops
Hmmm ... A person can not call a company , if you have no idea its name ( since the designer is unknown ... SUPPOSEDLY ) , and order a gown without a dress name or style number .

person
company
Hmmm ... A company can not call a person , if you have no idea its name ( since the designer is unknown ... SUPPOSEDLY ) , and order a gown without a dress name or style number .

company
person
The worst thing that can happen for any restaurant like Zahav is to have too many people write hyperbolic reviews making claims that " everyone " is going to " love " the food , decor and service .

people
reviews
The worst thing that can happen for any restaurant like Zahav is to have too many reviews write hyperbolic people making claims that " everyone " is going to " love " the food , decor and service .

reviews
people
The 

BNA
We live in the sub-division around the corner and after it happened and we brought it up at some neighborhood gatherings we discovered that several people from this neighborhood alone had pets go to Dr. Mann for surgical procedures and came back dead .

people
pets
We live in the sub-division around the corner and after it happened and we brought it up at some neighborhood gatherings we discovered that several pets from this neighborhood alone had people go to Dr. Mann for surgical procedures and came back dead .

pets
people
The menu had plenty of options even for picky eaters .

menu
plenty
The plenty had menu of options even for picky eaters .

plenty
menu
I googled Garage Door Repair in Woodinville and found NDI - Johnette answered the phone and was oh - so pleasant and helpful !

Johnette
phone
I googled Garage Door Repair in Woodinville and found NDI - phone answered the Johnette and was oh - so pleasant and helpful !

phone
Johnette
Kelly hit the nail on the head .

Kelly
na

reply
I have seen horses Loose Nails On there Own And do just fine with ONE nail gone Now if he had left out 2 or 3 I would be wondering to ...

horses
Nails
I have seen Nails Loose horses On there Own And do just fine with ONE nail gone Now if he had left out 2 or 3 I would be wondering to ...

Nails
horses
Our waiter quickly grasped the fact that I like Ginger ale and rather spicy foods while my wife 's taste ran to lemonade and less spicy fare .

waiter
fact
Our fact quickly grasped the waiter that I like Ginger ale and rather spicy foods while my wife 's taste ran to lemonade and less spicy fare .

fact
waiter
The ship offers variety of eatables , deliciously made .

ship
variety
The variety offers ship of eatables , deliciously made .

variety
ship
Other will add a drop of Methylene blue .

Other
drop
drop will add a Other of Methylene blue .

drop
Other
Ireland receives a lot of rain , with certain areas getting a soaking as many as 270 days of the year .

Ireland
lot
lot receive

Hens
Five Toes : Silkies have five toes .

Silkies
toes
Five Toes : toes have five Silkies .

toes
Silkies
Beards / Muffs : If your Silkie has a beard / muff the offspring could have a beard as well , as these genes are incomplete dominate .

offspring
beard
Beards / Muffs : If your Silkie has a beard / muff the beard could have a offspring as well , as these genes are incomplete dominate .

beard
offspring
Sussex have White Skin which could be recessive or dominate .

Sussex
Skin
Skin have White Sussex which could be recessive or dominate .

Skin
Sussex
Sussexs have soft feathers .

Sussexs
feathers
feathers have soft Sussexs .

feathers
Sussexs
Silkies have hookless feathers .

Silkies
feathers
feathers have hookless Silkies .

feathers
Silkies
The word renaissance ( Rinascimento in Italian ) means “ rebirth ” , and the era is best known for the renewed interest in the culture of classical antiquity after the period that Renaissance humanists labelled the Dark Ages .

word
rebirth
Th

slots
It is not as precise as a pelham bit because it only uses one rein , though the modern slots have two or three ones on which to put the reins .

slots
ones
scroll down and this website shows each thing you can buy (:

website
thing
scroll down and this thing shows each website you can buy (:

thing
website
Because Large Fries give you FOUR PIECES !

Fries
PIECES
Because Large PIECES give you FOUR Fries !

PIECES
Fries
There should be a Pixel Mapping function in the menu that will get rid of them , most sensors have a few dead and flipped pixels , some only show up on long exposures .

sensors
pixels
There should be a Pixel Mapping function in the menu that will get rid of them , most pixels have a few dead and flipped sensors , some only show up on long exposures .

pixels
sensors
The people abandoned their farms and religious sites and the ground they stood on became moorland .

people
farms
The farms abandoned their people and religious sites and the ground they stood on became

...

Police and prosecutors have the tools .

Police
tools
tools and prosecutors have the Police .

tools
Police
More than ever , police and prosecutors have the tools and the will to go after horrific crimes , even when the defendants are seemingly powerful individuals or institutions .

police
tools
More than ever , tools and prosecutors have the police and the will to go after horrific crimes , even when the defendants are seemingly powerful individuals or institutions .

tools
police
Like the tens of billions of dollars that have been " lost " in Iraq , planeloads of arms do n't just " vanish " ; not when the Pentagon contracts the work to an international criminal of Bout 's untouchable stature .

Pentagon
work
Like the tens of billions of dollars that have been " lost " in Iraq , planeloads of arms do n't just " vanish " ; not when the work contracts the Pentagon to an international criminal of Bout 's untouchable stature .

work
Pentagon
But that means nothing , since they were n't t

number
It is widely acknowledged that the Pakistani number has nurtured a ISI of extremist Islamist groups , such as the Harkat - ul - Jehad - al - Islami , in the port city of Chittagong , and put a number of secessionist rebels from India 's northeast in touch with this terrorist network .

number
ISI
Until 1995 the LTTE maintained a base at Twante , an island off the coat of Myanmar , west of the Andaman islands .

LTTE
base
Until 1995 the base maintained a LTTE at Twante , an island off the coat of Myanmar , west of the Andaman islands .

base
LTTE
In 1997 , the Thai navy reported the interception of a 16 - meter boat after a chase off the Thai port of Ranong , and the confiscation of two tons of weapons and ammunition .

navy
interception
In 1997 , the Thai interception reported the navy of a 16 - meter boat after a chase off the Thai port of Ranong , and the confiscation of two tons of weapons and ammunition .

interception
navy
As is LTTE standard procedure , the vessel changed 

The future Guard joined the president in May 1968 .

Guard
president
The F - 102 saw service in the Vietnam theater between March 1962 and December 1969 .

F
service
The service - 102 saw F in the Vietnam theater between March 1962 and December 1969 .

service
F
With Colin Powell around , the U.S. has no need of an Osama bin Laden

U.S.
need
With Colin Powell around , the need has no U.S. of an Osama bin Laden

need
U.S.
Once the Manmohan Singh government shows that it has little appetite for suicide , Musharraf will face the moment of truth : accept the inevitable , or once again ramp up the insurgency and spawn a fresh lot of killers that can hit not merely Mumbai and New Delhi but London and Chicago as well .

Musharraf
moment
Once the Manmohan Singh government shows that it has little appetite for suicide , moment will face the Musharraf of truth : accept the inevitable , or once again ramp up the insurgency and spawn a fresh lot of killers that can hit not merely Mumbai and New De

Edwards is a strong supporter of Israel , and believes that the U.S. has a vital role in promoting peace between the Israelis and the Palestinians . "

U.S.
role
Edwards is a strong supporter of Israel , and believes that the role has a vital U.S. in promoting peace between the Israelis and the Palestinians . "

role
U.S.
Musharraf made his coup in part because of the military 's anger over Prime Minister Nawaz Sharif 's willingness to back down from confronting India over Kashmir , so that he explicitly came to power as a warmonger .

Musharraf
coup
coup made his Musharraf in part because of the military 's anger over Prime Minister Nawaz Sharif 's willingness to back down from confronting India over Kashmir , so that he explicitly came to power as a warmonger .

coup
Musharraf
Not only did Bush not know who General Pervez Musharraf was , he seems to have confused coup - making with " taking office , " and moreover went on to suggest that the overthrow of an elected prime minister and

,condition,feature,id,object,predicted_value,subject,swapped?
0,subject,Vision,reviews-211797-0020\n,owners,4.491,shops,False
1,subject,Bright,reviews-211797-0020\n,owners,1.249,shops,False
2,subject,Dark,reviews-211797-0020\n,owners,0.069,shops,False
3,subject,Color,reviews-211797-0020\n,owners,0.786,shops,False
4,subject,Pattern,reviews-211797-0020\n,owners,1.138,shops,False


In [50]:
#tidy_df[tidy_df[id] =='reviews-211797-0020\n']#[tidy_df['feature']=='Vision']
print(tidy_df.columns)
#tidy_df.loc[(df['id'] == 'reviews-211797-0020\n') & (tidy_df['feature'] == 'Vision')]
tidy_df.loc[(tidy_df['id'] == 'reviews-211797-0020\n') & (tidy_df['feature'] == 'Vision')]

Index(['condition', 'feature', 'id', 'object', 'predicted_value', 'subject',
       'swapped?'],
      dtype='object')


,condition,feature,id,object,predicted_value,subject,swapped?
0,subject,Vision,reviews-211797-0020\n,owners,5.213,shops,False
62,object,Vision,reviews-211797-0020\n,owners,3.002,shops,False
124,subject,Vision,reviews-211797-0020\n,shops,3.002,owners,True
186,object,Vision,reviews-211797-0020\n,shops,5.213,owners,True


In [24]:
# convert to tody data format
import ast
df = pd.read_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')

feature_labels = list(features)

print(feature_labels)
print(len(feature_labels))

ids = []
sent = []
subject = []
obj = []
condition = []
swapped = []
feature = []
predicted_value = []

for index, row in df.iterrows():
    subject_predictions = ast.literal_eval(row.subject_predictions)
    object_predictions = ast.literal_eval(row.object_predictions)
    print(len(subject_predictions))
    i = 0
    for value in subject_predictions:
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("subject")
        feature = feature_labels[i]
        predicted_value = value
        i+=1
    
    i = 0
    for value in object_predictions:
        ids.append(row.id)
        subject.append(row.subject)
        obj.append(row.object)
        swapped.append(row['swapped?'])
        condition.append("object")
        feature = feature_labels[i]
        predicted_value = value
        i+=1

#cols = ["id", "subject", "object", "swapped?", "condition", "feature", "predicted_value"]
#pd.DataFrame([ids, subject, obj, swapped, feature, predicted_value], columns = cols)

tidy_df = pd.DataFrame.from_records(
    {"id": ids,
    "subject": subject, 
    "object": obj, 
    "swapped?": swapped, 
    "condition": condistion, 
    "feature": feature, 
    "predicted_value": predicted_value}
)
tidy_df.to_csv('../data/processed/swapped_subject_and_object_predictions_label_propagation.csv')


['Vision', 'Bright', 'Dark', 'Color', 'Pattern', 'Large', 'Small', 'Motion', 'Biomotion', 'Fast', 'Slow', 'Shape', 'Face', 'Body', 'Touch', 'Temperature', 'Texture', 'Weight', 'Pain', 'Audition', 'Loud', 'Low', 'High', 'Sound', 'Music', 'Speech', 'Taste', 'Smell', 'Head', 'UpperLimb', 'LowerLimb', 'Landmark', 'Path', 'Scene', 'Near', 'Toward', 'Away', 'Number', 'Time', 'Duration', 'Long', 'Short', 'Consequential', 'Social', 'Human', 'Communication', 'Self', 'Cognition', 'Benefit', 'Harm', 'Pleasant', 'Unpleasant', 'Happy', 'Sad', 'Angry', 'Disgusted', 'Fearful', 'Surprised', 'Drive', 'Needs', 'Attention', 'Arousal']
62


SyntaxError: invalid syntax (<unknown>, line 1)

In [13]:
"""
to get the features for the subject, select the dataframe sentences that aren't swapped and the prediction for the subject
"""
results = {}

subj = df[df['swapped?'] == False]
preds = subj.subject_predictions
average_subject_vector = np.average(preds, axis = 0)


subject_features = dict(zip(features,average_subject_vector))
results["original subject"] = subject_features




In [14]:
print(subj)

                                                         id                                                      \
0                                                                                         reviews-211797-0020\n   
2                                                                                         reviews-211797-0007\n   
4                                                                                         reviews-053248-0002\n   
6                                                                                         reviews-363685-0030\n   
8                                                                                         reviews-363685-0010\n   
10                                                                                        reviews-290594-0006\n   
12                                                                                        reviews-290594-0005\n   
14                                                                              

In [15]:
"""
to get the features for the object, select the dataframe sentences that aren't swapped and the prediction for the object
"""

obj = df[df['swapped?'] == False]
preds = obj.object_predictions
average_object_vector = np.average(preds, axis = 0)

object_features = dict(zip(features,average_object_vector))
results["original object"] =  object_features


In [16]:
"""
to get the features for the (original) subject in swapped position, select the dataframe sentences that are swapped and the prediction for the object
"""
subj = df[df['swapped?'] == True]
preds = subj.object_predictions
average_swapped_subject_vector = np.average(preds, axis = 0)

swapped_subject_features = dict(zip(features,average_swapped_subject_vector))
results["swapped subject"] = swapped_subject_features


In [17]:
"""
to get the features for the (original) onject in swapped position, select the dataframe sentences that are swapped and the prediction for the subject
"""

subj = df[df['swapped?'] == True]
preds = subj.subject_predictions
average_swapped_object_vector = np.average(preds, axis = 0)

swapped_object_features = dict(zip(features,average_swapped_object_vector))
results["swapped object"] = swapped_object_features


In [18]:

#print(results)
#pd.DataFrame.from_dict(results, orient='index')

graph = pd.DataFrame.from_records(results)

print(graph)



               original object  original subject  swapped object  \
Vision              3.377             4.214            3.556       
Bright              0.836             1.334            0.936       
Dark                0.611             0.798            0.704       
Color               1.134             2.067            1.347       
Pattern             1.161             1.679            1.305       
Large               1.708             1.938            1.775       
Small               1.048             1.252            1.095       
Motion              1.498             1.935            1.682       
Biomotion           1.447             2.248            1.672       
Fast                0.971             1.277            1.071       
Slow                0.669             0.825            0.727       
Shape               2.137             2.859            2.349       
Face                0.992             1.684            1.129       
Body                1.369             1.854     

In [19]:
graph = graph[['original subject', 'swapped subject', 'swapped object', 'original object']]
graph
#graph.columns.tolist()

,original subject,swapped subject,swapped object,original object
Vision,4.214,4.206,3.556,3.377
Bright,1.334,1.187,0.936,0.836
Dark,0.798,0.818,0.704,0.611
Color,2.067,1.889,1.347,1.134
Pattern,1.679,1.681,1.305,1.161
Large,1.938,1.895,1.775,1.708
Small,1.252,1.259,1.095,1.048
Motion,1.935,2.010,1.682,1.498
Biomotion,2.248,2.230,1.672,1.447
Fast,1.277,1.375,1.071,0.971


In [24]:
%matplotlib
# Try to visualize this in a nice way 
import seaborn as sns; sns.set_theme()

#sns.heatmap(graph.reindex(graph.mean().sort_values().index, axis=1), cmap="coolwarm",annot=True)

ax = sns.heatmap(graph, cmap="coolwarm", yticklabels=True)

ax.plot()

Using matplotlib backend: MacOSX


[]

In [21]:
# just look at subject and object
ax = sns.heatmap(graph[['original subject', 'original object']], cmap="coolwarm", yticklabels=True)

ax.plot()

[]

In [22]:
#graph.index.name = 'Feature'
#graph.index.name

In [23]:
ax = sns.lineplot(x = 'Feature', y = 'original subject', ci=None, data=graph)
ax.plot()

ValueError: Could not interpret value `Feature` for parameter `x`

In [ ]:
graph = graph.reset_index()
graph = graph.rename(columns={"index": 'Feature'})
graph

In [ ]:
graph['Feature'] = graph['Feature'].astype(str)

graph['Feature'] = pd.Categorical(graph['Feature'], categories=map(str, graph.Feature), ordered=True)


df2 = pd.melt(graph, 'Feature', var_name='Grammatical Position', 
              value_name='Value')
df2 = df2.astype({'Value':'float'})
df2

In [ ]:
# try to make a pretty line chart


sns.lineplot('Feature', 'Value', hue='Grammatical Position', data=df2)

### This is the Delta Analysis

In [ ]:
print(df.id.unique())

In [ ]:
len(eval_words)

In [ ]:
# subject analysis""""""

In [ ]:
deltas = []

for idd in df.id.unique():
    preds = df[df['id'] == idd]
    subj = preds[preds['swapped?'] == False]
    obj = preds[preds['swapped?'] == True]

    subj_avg = np.average(subj.predictions)
    obj_avg = np.average(obj.predictions)
    delta = subj_avg - obj_avg
    deltas.append(delta)
    
    #print(idd)
    #print(dict(zip(features,delta)))
    
print("average delta from subj to swapped subject across lemmas")
avg_delta = np.average(deltas, axis = 0)
print(dict(zip(features,avg_delta)))


In [ ]:
# so we want to take each category
coarse_categories = set(feature_categories.values())

In [ ]:
results = dict()
for coarse_cat in coarse_categories:
    print(coarse_cat)
    delta_results = []
    for delta in deltas:

        delta = dict(zip(features,delta))
        #print(delta)

        res = []
        for key, value in delta.items():
            if feature_categories[key] == coarse_cat:
                #print(coarse_cat)
                #print(value)
                res.append(value)
        #print(len(res))
        #print(res)
        res = np.average(res)
        #print(res)
        delta_results.append(res)
    results[coarse_cat] = np.average(delta_results)
print(results)

# construction analysis

hypothesis: nouns in ASNs (argument supporting constructions) will behave more like metaphorical language in emphasizing abstract features. and nouns in compounds will behave more liek literal uses, with emphasis of physical features

In [ ]:
# check that we've got this label coded correctly
eval_words.normalized_construction.unique()

In [ ]:
const_deltas = []

for lemma in eval_words.lemma.unique():
    preds = eval_words[eval_words['word'] == lemma]
    lit = preds[preds['normalized_construction'] == 'compound']
    met = preds[preds['normalized_construction'] == 'prep_complement']

    lit_avg = np.average(met.predictions)
    met_avg = np.average(lit.predictions)
    delta = lit_avg - met_avg
    const_deltas.append(delta)
    
    print(lemma)
    print(dict(zip(features,delta)))
    
print("average delta from compound (lit) to prep complement (met) across lemmas")
avg_delta = np.average(deltas, axis = 0)
print(dict(zip(features,avg)))

In [ ]:
results = dict()
for coarse_cat in coarse_categories:
    print(coarse_cat)
    delta_results = []
    for delta in const_deltas:

        delta = dict(zip(features,delta))
        #print(delta)

        res = []
        for key, value in delta.items():
            if feature_categories[key] == coarse_cat:
                #print(coarse_cat)
                #print(value)
                res.append(value)
        #print(len(res))
        #print(res)
        res = np.average(res)
        print(res)
        delta_results.append(res)
    results[coarse_cat] = np.average(delta_results)
#print(results)